In [1]:
# concreteModel version of warehouse location problem
import pyomo.environ as pyo


def create_warehouse_model(N, M, d, P):
    model = pyo.ConcreteModel(name="(WL)")

    model.x = pyo.Var(N, M, bounds=(0, 1))
    model.y = pyo.Var(N, within=pyo.Binary)

    # WL.1 equation
    def obj_rule(mdl):
        return sum(d[n, m] * mdl.x[n, m] for n in N for m in M)

    model.obj = pyo.Objective(rule=obj_rule)

    # WL.2 equation
    def demand_rule(mdl, m):
        return sum(mdl.x[n, m] for n in N) == 1

    model.demand = pyo.Constraint(M, rule=demand_rule)

    # WL.3 equation
    def warehouse_active_rule(mdl, n, m):
        return mdl.x[n, m] <= mdl.y[n]

    model.warehouse_active = pyo.Constraint(N, M, rule=warehouse_active_rule)

    # WL.4 equation
    def num_warehouses_rule(mdl):
        return sum(mdl.y[n] for n in N) <= P

    model.num_warehouses = pyo.Constraint(rule=num_warehouses_rule)

    return model


breaking_point = 0

In [2]:
# below could be changed to main.py

In [3]:
# Solve an instance of the WL problem
# Import Pyomo environment and model
import os
import pandas as pd
import pyomo.environ as pyo

os.environ['NEOS_EMAIL'] = 'hadjavo1@binghamton.edu'


# Establish the data for this model (this could also be
# imported using other Python packages)

# TODO - spin - read a data file
def cost_2_wcl(fpath):
    df = pd.read_excel(fpath, index_col=0)
    N = list(df.index.map(str))
    M = list(df.columns.map(str))

    d = {(n, m): df.at[n, m] for n in N for m in M}

    return N, M, d


breaking_point = 0


def main():
    dirpath = "./data/test/"
    files = [fname for fname in os.listdir(dirpath) if fname.endswith('.xlsx') and 'WLp' in fname]

    # min nbr of warehouses to be constructed
    P = 2

    for fName in files:
        N, M, d = cost_2_wcl(dirpath + fName)

        # Create the Pyomo model
        model = create_warehouse_model(N, M, d, P)

        # Create the solver interface and solve the model
        solver_manager = pyo.SolverManagerFactory('neos')
        res = solver_manager.solve(model, solver='cbc')
        model.display()


if __name__ == '__main__':
    main()
    breaking_point = 0


Model '(WL)'

  Variables:
    x : Size=12, Index=x_index
        Key                      : Lower : Value : Upper : Fixed : Stale : Domain
          ('Ashland', 'Chicago') :     0 :   1.0 :     1 : False : False :  Reals
          ('Ashland', 'Houston') :     0 :   0.0 :     1 : False : False :  Reals
               ('Ashland', 'LA') :     0 :   0.0 :     1 : False : False :  Reals
              ('Ashland', 'NYC') :     0 :   1.0 :     1 : False : False :  Reals
        ('Harlingen', 'Chicago') :     0 :   0.0 :     1 : False : False :  Reals
        ('Harlingen', 'Houston') :     0 :   1.0 :     1 : False : False :  Reals
             ('Harlingen', 'LA') :     0 :   1.0 :     1 : False : False :  Reals
            ('Harlingen', 'NYC') :     0 :   0.0 :     1 : False : False :  Reals
          ('Memphis', 'Chicago') :     0 :   0.0 :     1 : False : False :  Reals
          ('Memphis', 'Houston') :     0 :   0.0 :     1 : False : False :  Reals
               ('Memphis', 'LA') :     0